# Convalescent Plasma Data Processing

## Setup: Input Parameters and Variables

In [84]:
from io import StringIO
import csv
import pandas as pd
import os
from datetime import date, timedelta # Access to current date and time operations
# from datetime import datetime
from uuid import uuid4
import arcgis
import arcpy

## Utilities

In [85]:
def process_facilities(input_excel, 
                       input_sheet_name,
                       address_field,
                       city_field,
                       state_field,
                       zipcode_field,
                       output_table_path,
                       output_fc_path,
                       address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                       facility_field=None,
                       symbology_layer=None,
                      ):
    
    ### GOAL: To geocode excel sheet with sites for ARC CCP Process
    # Sites may correspond to Blood Collection Sites or Plasma Distribution Sites
    
    # Convert excel sheet to FGDB table
    print("Converting to table...")
    table = arcpy.conversion.ExcelToTable(input_excel, 
                                          output_table_path, 
                                          input_sheet_name).getOutput(0)
    
    # Geocode FGDB Table using address fields
    print("Converting to facilities...")
    facilities_fc = arcpy.geocoding.GeocodeAddresses(table, 
                                                     address_locator, 
                                                     f"Address {address_field} VISIBLE NONE;City {city_field} VISIBLE NONE;State {state_field} VISIBLE NONE;ZIP_Code {zipcode_field} VISIBLE NONE", 
                                                     output_fc_path, 
                                                     "STATIC", None, '', 
                                                     "Subaddress;'Point Address';'Street Address';'Distance Marker';Intersection;'Street Name';'Primary Postal';'Postal Locality';'Postal Extension'").getOutput(0)
    
    # Add and calculate Facility_ID field
    print("Creating Facility ID field...")
    if facility_field:
        arcpy.management.CalculateField(facilities_fc, "FACILITY_ID", f"!{facility_field}!", "PYTHON3", '', "LONG")
    else:
        arcpy.management.CalculateField(facilities_fc, "FACILITY_ID", "!ObjectID!", "PYTHON3", '', "LONG")
    
    return facilities_fc

In [86]:
def process_incidents(input_excel, 
                      input_sheet_name,
                      address_field,
                      city_field,
                      state_field,
                      zipcode_field,
                      output_table_path,
                      output_fc_path,
                      address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                      incident_field=None,
                      symbology_layer=None):
    
    ### GOAL: To geocode excel sheet with incidents for ARC CCP Process
    # Incidents may correspond to Donor Leads or Plasma Need Locations
    
    # Convert excel sheet to FGDB table
    print("Converting to table...")
    table = arcpy.conversion.ExcelToTable(input_excel, 
                                          output_table_path, 
                                          input_sheet_name).getOutput(0)
    
    # Geocode FGDB Table using address fields
    print("Converting to incidents...")
    incidents_fc = arcpy.geocoding.GeocodeAddresses(table, 
                                                    address_locator, 
                                                    f"Address {address_field} VISIBLE NONE;City {arcpy.ValidateFieldName(city_field)} VISIBLE NONE;State {arcpy.ValidateFieldName(state_field)} VISIBLE NONE;ZIP_Code {arcpy.ValidateFieldName(zipcode_field)} VISIBLE NONE", 
                                                    output_fc_path, 
                                                    "STATIC", None, '', 
                                                    "Subaddress;'Point Address';'Street Address';'Distance Marker';Intersection;'Street Name';'Primary Postal';'Postal Locality';'Postal Extension'").getOutput(0)
    
    # Add and calculate Facility_ID field
    print("Creating Incident ID field...")
    if incident_field:
        arcpy.management.CalculateField(incidents_fc, "INCIDENT_ID", f"!{incident_field}!", "PYTHON3", '', "LONG")
    else:
        arcpy.management.CalculateField(incidents_fc, "INCIDENT_ID", "!ObjectID!", "PYTHON3", '', "LONG")
    
    return incidents_fc

In [87]:
def process_leads_csv(input_csv, 
                      address_field,
                      city_field,
                      state_field,
                      zipcode_field,
                      country_field,
                      workspace_fgdb,
                      output_table_name,
                      output_fc_path,
                      address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                      incident_field=None,
                      symbology_layer=None,
                      process_cohort_field=True,
                      cohort_field="Likely",
                      process_US_only=True
                     ):
    
    ### GOAL: To geocode csv with incidents for ARC CCP Process
    # Incidents may correspond to Donor Leads or Plasma Need Locations
    
    # Use csv reader to find csv header row
    print("Finding header row...")
    with open(input_csv) as fin:
        reader = csv.reader(fin)
        idx = next(idx for idx, row in enumerate(reader) if len(row)>1)
        
    # Use pandas to pre-process the csv
    df = pd.read_csv(input_csv, skiprows=idx)
    # If we need to convert non-"Yes" values to "No", perform the change
    print("\nCohort field value counts:")
    print(df[cohort_field].value_counts())
    if process_cohort_field:
        print("Processing cohort field...")
        df[cohort_field].mask(df[cohort_field] != "Yes", "No", inplace=True)
        print("New cohort value counts:")
        print(df[cohort_field].value_counts())
    
    print("\nCountry values:")
    print(df[country_field].value_counts())
    if process_US_only:
        print("Using United States records...")
        df = df.loc[df[country_field] == "United States"]
    
    print("\nExporting to local temp CSV...")
    temp_csv_name = "temp_{0}.csv".format(str(uuid4()))
    df.to_csv(temp_csv_name)

    # Convert excel sheet to FGDB table
    print("Converting to table...")
    table = arcpy.conversion.TableToTable(temp_csv_name, workspace_fgdb, output_table_name).getOutput(0)
    
    print("Removing temp CSV...")
    os.remove(temp_csv_name)
    
    # Geocode FGDB Table using address fields
    print("Converting to incidents...")
    incidents_fc = arcpy.geocoding.GeocodeAddresses(table, 
                                                    address_locator, 
                                                    f"Address {address_field} VISIBLE NONE;City {arcpy.ValidateFieldName(city_field)} VISIBLE NONE;State {arcpy.ValidateFieldName(state_field)} VISIBLE NONE;ZIP_Code {arcpy.ValidateFieldName(zipcode_field)} VISIBLE NONE", 
                                                    output_fc_path, 
                                                    "STATIC", None, '', 
                                                    "Subaddress;'Point Address';'Street Address';'Distance Marker';Intersection;'Street Name';'Primary Postal';'Postal Locality';'Postal Extension'").getOutput(0)
    
    # Add and calculate Facility_ID field
    print("Creating Incident ID field...")
    if incident_field:
        arcpy.management.CalculateField(incidents_fc, "INCIDENT_ID", f"!{incident_field}!", "PYTHON3", '', "LONG")
    else:
        arcpy.management.CalculateField(incidents_fc, "INCIDENT_ID", "!ObjectID!", "PYTHON3", '', "LONG")
    
    return incidents_fc

In [88]:
def process_connections(input_facilities_fc, 
                        input_incidents_fc,
                        output_fgdb,
                        output_connections_fc_name,
                        routing_nd, 
                        direction="TO_FACILITIES", # Alternative: "FROM_FACILITIES"
                       ):
    
    ### GOAL: To run network analysis between provided facilities and incidents for ARC CPP Process
    # Facilities and incidents can be part of Phase 1 (Donor Leads to Blood Collection Sites), or
    # Phase 2 (Distribution Sites to Care-givers with Need)
    
    # Make Closest Facility layer, using local road network
    print("Creating NA Object...")
    closest_facility_obj = arcpy.na.MakeClosestFacilityAnalysisLayer(routing_nd, 
                                                                     "ClosestFacility_{0}".format(date.today().strftime("%Y%m%d")),
                                                                     "Driving Time", "TO_FACILITIES", 
                                                                     None, 1, None, "LOCAL_TIME_AT_LOCATIONS", "START_TIME", 
                                                                     "ALONG_NETWORK", "Miles;TravelTime;WalkTime", "NO_DIRECTIONS")
    
    # Add Facilities
    print("Adding Facilities...")
    arcpy.na.AddLocations(closest_facility_obj, 
                          "Facilities", 
                          input_facilities_fc, 
                          "Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_TravelTime # 0;Attr_Miles # 0;Attr_Kilometers # 0;Attr_TimeAt1KPH # 0;Attr_WalkTime # 0;Attr_TruckMinutes # 0;Attr_TruckTravelTime # 0;Cutoff_Minutes # #;Cutoff_TravelTime # #;Cutoff_Miles # #;Cutoff_Kilometers # #;Cutoff_TimeAt1KPH # #;Cutoff_WalkTime # #;Cutoff_TruckMinutes # #;Cutoff_TruckTravelTime # #", 
                          "5000 Meters", None, 
                          "Routing_Streets SHAPE;Routing_Streets_Override NONE;Routing_ND_Junctions NONE", 
                          "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", 
                          "Routing_Streets #;Routing_Streets_Override #;Routing_ND_Junctions #")
    
    # Add Incidents
    print("Adding Incidents...")
    arcpy.na.AddLocations(closest_facility_obj, 
                          "Incidents", 
                          input_incidents_fc, 
                          "Name # #;TargetFacilityCount # #;CurbApproach # 0;Attr_Minutes # 0;Attr_TravelTime # 0;Attr_Miles # 0;Attr_Kilometers # 0;Attr_TimeAt1KPH # 0;Attr_WalkTime # 0;Attr_TruckMinutes # 0;Attr_TruckTravelTime # 0;Cutoff_Minutes # #;Cutoff_TravelTime # #;Cutoff_Miles # #;Cutoff_Kilometers # #;Cutoff_TimeAt1KPH # #;Cutoff_WalkTime # #;Cutoff_TruckMinutes # #;Cutoff_TruckTravelTime # #", 
                          "5000 Meters", None, 
                          "Routing_Streets SHAPE;Routing_Streets_Override NONE;Routing_ND_Junctions NONE", 
                          "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", 
                          "Routing_Streets #;Routing_Streets_Override #;Routing_ND_Junctions #")

    # Solve
    print("Solving...")
    arcpy.na.Solve(closest_facility_obj, "SKIP", "TERMINATE", None, '')  
    
    # Copy output routes to designated output fc
    print("Exporting connections to disk...")
    layer_object = closest_facility_obj.getOutput(0)
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    print(sublayer_names)
#     connections_layer = sublayer_names["CFRoutes"]
#     arcpy.conversion.FeatureClassToFeatureClass(connections_layer, 
#                                                 output_fgdb, 
#                                                 output_connections_fc_name)
    connections_fc = arcpy.conversion.FeatureClassToFeatureClass("ClosestFacility_{0}/Routes".format(date.today().strftime("%Y%m%d")), 
                                                                 output_fgdb, 
                                                                 output_connections_fc_name).getOutput(0)
    
    return connections_fc

In [89]:
def post_process_connections_with_lead_info(connections_fc, leads_fc, 
                                            connections_id_field="IncidentID", leads_incident_id_field="ObjectID",
                                            cohort_field_name="Likely"):
    
    ### GOAL: To add the incident cohort field name to the connections layer so 
    # we can query network connections by cohort value on the front-end
    
    # Sites may correspond to Blood Collection Sites or Plasma Distribution Sites
    
    print("Adding join...")
    arcpy.management.AddJoin(connections_fc, connections_id_field, leads_fc, leads_incident_id_field, "KEEP_ALL")
    
    print("Calculating ID field...")
    new_field_name = "Likely"
    source_field_name = "{}.USER_Likely".format(str(leads_fc.split('\\')[-1]))
    arcpy.management.CalculateField(connections_fc, "Likely", f"!{source_field_name}!", "PYTHON3", '', "TEXT")
    print("Removing join...")
    arcpy.management.RemoveJoin(connections_fc, str(leads_fc.split('\\')[-1]))
    print("Step complete.")

In [90]:
def post_process_facilities_with_connection_info(connections_fc,
                                                 facilities_fc,
                                                 output_summary_table,
                                                 output_fgdb,
                                                 output_facilities_fc_name,
                                                 case_field="FacilityID",
                                                 facilities_id_field="FACILITY_ID"
                                                ):
    
    ### GOAL: To add the connection statistics for each facility for each cohort of donor
    print("Creating summary stats...")
    stats_table = arcpy.analysis.Statistics(connections_fc, output_summary_table, 
                                            "Total_TravelTime MEAN;Total_TravelTime MEDIAN;Total_TravelTime MAX;Total_TravelTime MIN;Total_TravelTime RANGE;Total_TravelTime STD;Total_Miles MEAN;Total_Miles MEDIAN;Total_Miles MAX;Total_Miles MIN;Total_Miles RANGE;Total_Miles STD", 
                                            case_field).getOutput(0)
    print("Joining tables...")
    arcpy.management.AddJoin(facilities_fc, facilities_id_field, stats_table, case_field, "KEEP_ALL")
    
    print("Exporting output...")
    out_facilities_fc = arcpy.conversion.FeatureClassToFeatureClass(facilities_fc, 
                                                                    output_fgdb, 
                                                                    output_facilities_fc_name).getOutput(0)
        
    return out_facilities_fc

In [91]:
def post_process_incidents_with_facilities(leads_fc, connections_fc, 
                                           output_fgdb, output_leads_fc_name,
                                           leads_id_field="INCIDENT_ID", connections_fc_field="IncidentID"
                                          ):
    
    # Set attribute indices
    print("Setting attribute indices...")
    arcpy.management.AddIndex(leads_fc, leads_id_field, "ID_Index", "UNIQUE", "ASCENDING")
    arcpy.management.AddIndex(connections_fc, connections_fc_field, "ID_Index", "UNIQUE", "ASCENDING")
    
    # Join connections to incidents
    print("Joining tables...")
    arcpy.management.AddJoin(leads_fc, leads_id_field, connections_fc, connections_fc_field, "KEEP_ALL")
    
    print("Exporting output...")
    out_leads_fc = arcpy.conversion.FeatureClassToFeatureClass(leads_fc, 
                                                               output_fgdb, 
                                                               output_leads_fc_name).getOutput(0)
    
    return leads_fc

In [92]:
def apply_symbology():
    arcpy.management.ApplySymbologyFromLayer(r"4 20 Data Update\FC_Leads", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\Blood Collection Sites- Not Selected.lyrx", None, "MAINTAIN")
    arcpy.management.ApplySymbologyFromLayer(r"4 20 Data Update\FC_Leads", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\Blood Collection Sites- Not Selected.lyrx", None, "MAINTAIN")
    arcpy.management.ApplySymbologyFromLayer(r"4 20 Data Update\FC_Leads", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\Blood Collection Sites- Not Selected.lyrx", None, "MAINTAIN")
    pass

In [93]:
def process_eap_sites_csv(input_csv,
                          id_field,
                          address_field,
                          city_field,
                          state_field,
                          zipcode_field,
                          workspace_fgdb,
                          output_table_name,
                          output_fc_path,
                          address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc"
                         ):
    
    ### GOAL: To geocode csv with incidents for ARC CCP Process
    # Incidents may correspond to Donor Leads or Plasma Need Locations
    
    # Use csv reader to find csv header row
    print("Finding header row...")
    with open(input_csv) as fin:
        reader = csv.reader(fin)
        idx = next(idx for idx, row in enumerate(reader) if len(row)>1)
        
    # Use pandas to pre-process the csv
    df = pd.read_csv(input_csv, skiprows=idx)
    
    print("Exporting to local temp CSV...")
    temp_csv_name = "temp_{0}.csv".format(str(uuid4()))
    df.to_csv(temp_csv_name)
        
    # Convert excel sheet to FGDB table
    print("Converting to table...")
    table = arcpy.conversion.TableToTable(temp_csv_name, workspace_fgdb, output_table_name).getOutput(0)
    
    print("Removing temp CSV...")
    os.remove(temp_csv_name)
    
    # Geocode FGDB Table using address fields
    print("Converting to feature class...")
    need_fc = arcpy.geocoding.GeocodeAddresses(table, 
                                               address_locator, 
                                               f"Address {address_field} VISIBLE NONE;City {arcpy.ValidateFieldName(city_field)} VISIBLE NONE;State {arcpy.ValidateFieldName(state_field)} VISIBLE NONE;ZIP_Code {arcpy.ValidateFieldName(zipcode_field)} VISIBLE NONE", 
                                               output_fc_path, 
                                               "STATIC", None, '', 
                                               "Subaddress;'Point Address';'Street Address';'Distance Marker';Intersection;'Street Name';'Primary Postal';'Postal Locality';'Postal Extension'").getOutput(0)
        
    return need_fc

In [121]:
def process_eapsites_demand(in_sites_fc_path, in_sites_id_field, in_needs_fc_path, in_needs_id_field, out_need_count_field_name):
    
    # Create the out_need_count field if it doesn't already exist
    if len(arcpy.ListFields(in_sites_fc_path, out_need_count_field_name)) == 0:
        print("Adding output field {}...".format(out_need_count_field_name))
        arcpy.AddField_management(in_sites_fc_path, out_need_count_field_name, "LONG")
    else:
        print("Using existing field {}...".format(out_need_count_field_name))

    need_sedf = pd.DataFrame.spatial.from_featureclass(in_needs_fc_path)

    # For each site
    with arcpy.da.UpdateCursor(in_sites_fc_path, [in_sites_id_field, out_need_count_field_name]) as cursor:
        for row in cursor:
            print(f"Current ID: {str(row[0])}")

            # Query for current site ID
            eap_count = need_sedf.loc[need_sedf[in_needs_id_field] == row[0]].shape[0]

            # Write count to site ID entry
            row[1] = eap_count
            cursor.updateRow(row)

    print("Completed.")

In [122]:
in_sites_fc_path = "C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\FC_EAPSites_200421"
in_sites_id_field = "USER_ExternalRefEntryID"
in_needs_fc_path = "C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\FC_Need_200421"
in_needs_id_field = "USER_ExternalRefID"
out_need_count_field_name = "EAP_Count"

process_eapsites_demand(in_sites_fc_path, in_sites_id_field, in_needs_fc_path, in_needs_id_field, out_need_count_field_name)

Using existing field EAP_Count...
Current ID: ARC006E63ALVa
Current ID: ARC036B1ANMEa
Current ID: ARC009N8FTCLa
Current ID: ARC019060800a
Current ID: ARC055B8DESOa
Current ID: ARC049T0300Ja
Current ID: ARC022XBBa
Current ID: ARC049T0046Ta
Current ID: ARC036B3BONSa
Current ID: ARC001B2BROKa
Current ID: ARC022BYMa
Current ID: ARC006B2CALIa
Current ID: ARC022XHFa
Current ID: ARC0122395a
Current ID: ARC0122397a
Current ID: ARC0122360a
Current ID: ARC007000100a
Current ID: ARC022NJCNSTa
Current ID: ARC0337007a
Current ID: ARC053B1CHILa
Current ID: ARC011B1CHNEa
Current ID: ARC022NNJCLAa
Current ID: ARC006B6CPDHa
Current ID: ARC038B3BRYNa
Current ID: ARC022CTRa
Current ID: ARC041A48010a
Current ID: ARC022EMMa
Current ID: ARC006PZ0932a
Current ID: ARC038B7ELKHa
Current ID: ARC017NM017a
Current ID: ARC017WYOMa
Current ID: ARC017PRINa
Current ID: ARC017RIDGa
Current ID: ARC017FVSDa
Current ID: ARC036B1GEORa
Current ID: ARC001B2GLENa
Current ID: ARC001B7059Ha
Current ID: ARC040B0GOTTa
Current ID

Current ID: MC534
Current ID: MC532
Current ID: MC531
Current ID: MC529
Current ID: MC494
Current ID: MC50
Current ID: MC492
Current ID: MC491
Current ID: MC493
Current ID: MC480
Current ID: MC476
Current ID: MC471
Current ID: MC468
Current ID: MC467
Current ID: MC488
Current ID: MC456
Current ID: MC43
Current ID: MC45
Current ID: MC420
Current ID: MC419
Current ID: MC421
Current ID: MC409
Current ID: MC413
Current ID: MC408
Current ID: MC403
Current ID: MC400
Current ID: MC398
Current ID: MC404
Current ID: MC397
Current ID: MC393
Current ID: MC395
Current ID: MC390
Current ID: MC388
Current ID: MC389
Current ID: MC379
Current ID: MC378
Current ID: MC376
Current ID: MC386
Current ID: MC367
Current ID: MC356
Current ID: MC35
Current ID: MC346
Current ID: MC345
Current ID: MC355
Current ID: MC34
Current ID: MC344
Current ID: MC336
Current ID: MC320
Current ID: MC316
Current ID: MC322
Current ID: MC305
Current ID: MC30
Current ID: MC296
Current ID: MC293
Current ID: MC292
Current ID: MC29

In [82]:
def process_needs_cumulative_time_table(fc_path, date_field, site_field, output_fc_path):
    
    def date_range(start, end):
        r = (end+datetime.timedelta(days=1)-start).days
        return [start+datetime.timedelta(days=i) for i in range(r)]
    
    # Import FC_Need into SEDF
    need_sedf = pd.DataFrame.spatial.from_featureclass(fc_path)

    # Determine first date
    start_date = need_sedf[date_field].sort_values().tolist()[0]

    # Determine last date
    last_date = need_sedf[date_field].sort_values().tolist()[-1]

    # Create list of days
    days_list = date_range(start_date, last_date)
    days_list

    # Create list of sites
    sites_list = need_sedf[site_field].unique().tolist()

    # Create new SEDF to receive each day's records
    new_sedf = need_sedf[[site_field, "SHAPE"]].iloc[0:1].copy()
    new_sedf = new_sedf.drop([0,0])
    new_sedf['New_Day_Count'] = 0
    new_sedf['Cumulative_Count'] = 0

    # For each site
    for site in sites_list:
        print("Processing site, {0}...".format(str(site)))

        # Get shape of Site
        site_shape = need_sedf.loc[need_sedf[site_field] == site].iloc[0]['SHAPE']
        print(site_shape)

        # Set variable for cumulative_counter
        cumulative_count = 0

        # For each day
        for day in days_list:

            # Query records for the site, for the day
            day_start = datetime.datetime.combine(day, datetime.datetime.min.time())
            day_end = datetime.datetime.combine(day, datetime.datetime.max.time())
            mask = (need_sedf[date_field] > day_start) & (need_sedf[date_field] <= day_end) & (need_sedf[site_field] == site)
            day_count = need_sedf.loc[mask].shape[0]
            if not day_count:
                day_count = 0

            cumulative_count += day_count

            # Write new row
            new_sedf = new_sedf.append({site_field: site, 
                                        'SHAPE': site_shape,
                                        'Date': day,
                                        'New_Day_Count': day_count, 
                                        'Cumulative_Count': cumulative_count}, ignore_index=True)

    print("Processing complete.")
    pass

In [80]:
fc_path = "C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\FC_Need_200421"
date_field = "USER_SubmissionDate"
site_field = "USER_ExternalRefID"

# Import FC_Need into SEDF
need_sedf = pd.DataFrame.spatial.from_featureclass(fc_path)

# Determine first date
start_date = need_sedf['USER_SubmissionDate'].sort_values().tolist()[0]

# Determine last date
last_date = need_sedf['USER_SubmissionDate'].sort_values().tolist()[-1]

# Create list of days
days_list = date_range(start_date, last_date)
days_list

# Create list of sites
sites_list = need_sedf[site_field].unique().tolist()

# Create new SEDF to receive each day's records
new_sedf = need_sedf[[site_field, "SHAPE"]].iloc[0:1].copy()
new_sedf = new_sedf.drop([0,0])
new_sedf['New_Day_Count'] = 0
new_sedf['Cumulative_Count'] = 0

# For each site
for site in sites_list:
    print("Processing site, {0}...".format(str(site)))
    
    # Get shape of Site
#     site_mask = (need_sedf[site_field] == site)
#     site_records = need_sedf.loc[mask]
#     need_sedf.loc[need_sedf[site_field] == site]
#     print(site_records)
    
#     site_shape = site_records.iloc[0]['SHAPE']
    site_shape = need_sedf.loc[need_sedf[site_field] == site].iloc[0]['SHAPE']
    print(site_shape)

    # Set variable for cumulative_counter
    cumulative_count = 0

    # For each day
    for day in days_list:
#         print("Processing day, {0}".format(str(day)))
        
        # Query records for the site, for the day
        day_start = datetime.datetime.combine(day, datetime.datetime.min.time())
        day_end = datetime.datetime.combine(day, datetime.datetime.max.time())
        mask = (need_sedf[date_field] > day_start) & (need_sedf[date_field] <= day_end) & (need_sedf[site_field] == site)
        day_count = need_sedf.loc[mask].shape[0]
        if not day_count:
#             print("Day count was null")
            day_count = 0
#         else:
# #             print("Day count: {0}".format(str(day_count)))
        cumulative_count += day_count
    
        # Write new row
        new_sedf = new_sedf.append({site_field: site, 
                                    'SHAPE': site_shape,
                                    'Date': day,
                                    'New_Day_Count': day_count, 
                                    'Cumulative_Count': cumulative_count}, ignore_index=True)
#         new_row = pd.Series({site_field: site, 'New_Day_Count': day_count, 'Cumulative_Count': cumulative_count})
#         new_sedf = pd.concat([new_sedf, new_row], ignore_index=True)

print("Processing complete.")

Processing site, ARC001003102a...
{'x': -75.93594862642715, 'y': 42.09271247919867, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count: 1
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC001B1SAMAa...
{'x': -73.67129562609148, 'y': 42.73400778153501, 'spatialRefe

Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count: 1
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count: 1
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC006B2CALIa...
{'x': -118.26548529421024, 'y': 34.03708770173711, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count: 1
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:

Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count: 1
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC022KIMa...
{'x': -74.21742510028736, 'y': 40.07389618296355, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-1

Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count: 1
Processing day, 2020-04-16 17:07:00
Day count: 1
Processing site, ARC027B7VACLa...
{'x': -80.36142422134992, 'y': 39.27033993783897, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count w

Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC040B0RUSHa...
{'x': -87.67115054196319, 'y': 41.874148099256445, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count: 1
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC040N35WMHa...
{'x': 

...

Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count: 1
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, ARC036B1BAPTa...
{'x': -81.03872655818333, 'y': 34.00577375685447, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day c

Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count: 2
Processing site, MC1159...
{'x': -98.49230831617513, 'y': 29.433175442393804, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count w

Processing day, 2020-04-14 17:07:00
Day count: 1
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, MC1732...
{'x': -74.06427427183934, 'y': 40.35319151987386, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count wa

Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count: 1
Processing day, 2020-04-16 17:07:00
Day count was null
Processing site, MC408...
{'x': -95.86362738369604, 'y': 36.06133574035323, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:

Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was null
Processing day, 2020-04-12 17:07:00
Day count was null
Processing day, 2020-04-13 17:07:00
Day count was null
Processing day, 2020-04-14 17:07:00
Day count was null
Processing day, 2020-04-15 17:07:00
Day count was null
Processing day, 2020-04-16 17:07:00
Day count: 1
Processing site, MC684...
{'x': -117.16793335623868, 'y': 32.6942470768422, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}
Processing day, 2020-04-03 17:07:00
Day count was null
Processing day, 2020-04-04 17:07:00
Day count was null
Processing day, 2020-04-05 17:07:00
Day count was null
Processing day, 2020-04-06 17:07:00
Day count was null
Processing day, 2020-04-07 17:07:00
Day count was null
Processing day, 2020-04-08 17:07:00
Day count was null
Processing day, 2020-04-09 17:07:00
Day count was null
Processing day, 2020-04-10 17:07:00
Day count was null
Processing day, 2020-04-11 17:07:00
Day count was

In [75]:
new_sedf

,USER_ExternalRefID,SHAPE,New_Day_Count,Cumulative_Count,Date
0,ARC001003102a,"{'x': -75.93594862642715, 'y': 42.092712479198...",0,0,2020-04-03 17:07:00
1,ARC001003102a,"{'x': -75.93594862642715, 'y': 42.092712479198...",0,0,2020-04-04 17:07:00
2,ARC001003102a,"{'x': -75.93594862642715, 'y': 42.092712479198...",0,0,2020-04-05 17:07:00
3,ARC001003102a,"{'x': -75.93594862642715, 'y': 42.092712479198...",0,0,2020-04-06 17:07:00
4,ARC001003102a,"{'x': -75.93594862642715, 'y': 42.092712479198...",0,0,2020-04-07 17:07:00
...,...,...,...,...,...
4559,MC967,"{'x': -97.0668425358536, 'y': 47.8879338479698...",0,1,2020-04-12 17:07:00
4560,MC967,"{'x': -97.0668425358536, 'y': 47.8879338479698...",0,1,2020-04-13 17:07:00
4561,MC967,"{'x': -97.0668425358536, 'y': 47.8879338479698...",0,1,2020-04-14 17:07:00
4562,MC967,"{'x': -97.0668425358536, 'y': 47.8879338479698...",0,1,2020-04-15 17:07:00


In [78]:
new_sedf.spatial.to_featureclass(r"C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\cumulative_table")

'C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\cumulative_table'

In [79]:
new_sedf.spatial.to_featureclass(r"C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\cumulative_table_2")

'C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\cumulative_table_2'

#### TODO: Connection Summary stats by COHORT

## Processing

### Phase 1: Donor Leads and Blood Collection Sites

In [94]:
# Leads Processing for 4/20/20
LEADS_input_csv = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\ESRI - Leads.csv"
LEADS_address_field = "Address"
LEADS_city_field = "City"
LEADS_state_field = r"State_Province_Region"
LEADS_zipcode_field = "Zipcode1"
LEADS_country_field = "Country1"
LEADS_workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
LEADS_output_table_name = "Table_Leads_200422"
LEADS_output_fc_path = LEADS_workspace_fgdb+r"\FC_Leads_200422"

leads_fc = process_leads_csv(LEADS_input_csv, 
                             LEADS_address_field,
                             LEADS_city_field,
                             LEADS_state_field,
                             LEADS_zipcode_field,
                             LEADS_country_field,
                             LEADS_workspace_fgdb,
                             LEADS_output_table_name,
                             LEADS_output_fc_path,
                             address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                             incident_field=None,
                             symbology_layer=None,
                             process_cohort_field=True,
                             cohort_field="Likely",
                             process_US_only=True)

Finding header row...

Cohort field value counts:
No              18071
Yes             17464
I don't know     1133
Name: Likely, dtype: int64
Processing cohort field...
New cohort value counts:
No     19204
Yes    17464
Name: Likely, dtype: int64

Country values:
United States           36548
Canada                     58
United Kingdom             34
India                       4
Spain                       4
France                      3
Philippines                 3
Mexico                      2
Iraq                        2
Portugal                    1
Japan                       1
New Zealand                 1
South Korea                 1
Australia                   1
Bangladesh                  1
Israel                      1
Jersey                      1
United Arab Emirates        1
Argentina                   1
Name: Country1, dtype: int64
Using United States records...

Exporting to local temp CSV...
Converting to table...
Removing temp CSV...
Converting to incidents...
Cr

In [95]:
# Collection Site processing for 4/20/2020

input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Archived\Phase1_ARC_Sample_File_3.xlsx"
input_sheet_name = "CollectionSites"
address_field = "AddressLine1"
city_field = "City"
state_field = "State"
zipcode_field = "ZipCode"
output_facilities_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_CollectionSites_200420"
output_facilities_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_CollectionSites_200420"

facilities_fc = process_facilities(input_excel, 
                                   input_sheet_name,
                                   address_field,
                                   city_field,
                                   state_field,
                                   zipcode_field,
                                   output_facilities_table_path,
                                   output_facilities_fc_path,
                                   address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                                   facility_field=None,
                                   symbology_layer=None)

Converting to table...
Converting to facilities...
Creating Facility ID field...


In [96]:
# Leads to BCS Connections
workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_connections_name = "Connections_LeadstoBCS_200422"
routing_nd = r"C:\ArcGIS\Business Analyst\US_2019\Data\Streets Data\NorthAmerica.gdb\Routing\Routing_ND"

connections_fc = process_connections(facilities_fc, 
                                     leads_fc,
                                     workspace_fgdb,
                                     output_connections_name,
                                     routing_nd)

Creating NA Object...
Adding Facilities...
Adding Incidents...
Solving...
Exporting connections to disk...
{'Facilities': 'Facilities', 'Incidents': 'Incidents', 'Barriers': 'Point Barriers', 'CFRoutes': 'Routes', 'PolylineBarriers': 'Line Barriers', 'PolygonBarriers': 'Polygon Barriers'}


In [ ]:
connections_fc

In [97]:
# Post-process Connections
post_process_connections_with_lead_info(output_connections_name, leads_fc)

Adding join...
Calculating ID field...
Removing join...
Step complete.


In [98]:
# Post-process BCS Summary Stats
# connections_fc = "Connections_Leads_to_BCS_200420"
# facilities_fc = "FC_CollectionSites_200420"
# leads_fc = "FC_Leads_200420"
output_summary_table = "in_memory\SummaryStats"
output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_facilities_fc_name = "FC_CollectionSites_200422_Processed"

post_process_facilities_with_connection_info(connections_fc,
                                             facilities_fc,
                                             output_summary_table,
                                             output_fgdb,
                                             output_facilities_fc_name)

Creating summary stats...
Joining tables...
Exporting output...


'C:\\Users\\albe9057\\Documents\\ArcGIS\\Projects\\ARC_Convalescent_Plasma\\ARC_Convalescent_Plasma.gdb\\FC_CollectionSites_200422_Processed'

In [99]:
# Post-process Leads Summary Stats
connections_fc = "Connections_LeadstoBCS_200422"
leads_fc = "FC_Leads_200422"
output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_leads_fc_name = "FC_Leads_200422_Processed"
leads_fc = post_process_incidents_with_facilities(leads_fc, connections_fc, output_fgdb, output_leads_fc_name)

Setting attribute indices...
Joining tables...
Exporting output...


### Phase 2: Care Givers and Requests

In [ ]:
workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
NEED_csv = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\ESRI - Demand Need.csv"
NEED_id_field = "ExternalRefID"
NEED_address_field = "Address"
NEED_city_field = "City"
NEED_state_field = "State"
NEED_zipcode_field = "ZipCode"
NEED_output_table_name = "Table_EAPNeed_200421"
NEED_output_fc_path = workspace_fgdb + r"\FC_EAPNeed_200421"

need_fc = process_eap_sites_csv(NEED_csv,
                                NEED_id_field,
                                NEED_address_field,
                                NEED_city_field,
                                NEED_state_field,
                                NEED_zipcode_field,
                                workspace_fgdb,
                                NEED_output_table_name,
                                NEED_output_fc_path,
                                address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc")

In [ ]:
workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
EAPSITES_csv = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\ESRI - Demand Sites.csv"
EAPSITES_id_field = "ExternalRefEntryID"
EAPSITES_address_field = "Address"
EAPSITES_city_field = "City"
EAPSITES_state_field = "State"
EAPSITES_zipcode_field = "ZipCode"
EAPSITES_output_table_name = "Table_EAPSites_200421"
EAPSITES_output_fc_path = workspace_fgdb + r"\FC_EAPSites_200421"

EAPSITES_fc = process_eap_sites_csv(EAPSITES_csv,
                                    EAPSITES_id_field,
                                    EAPSITES_address_field,
                                    EAPSITES_city_field,
                                    EAPSITES_state_field,
                                    EAPSITES_zipcode_field,
                                    workspace_fgdb,
                                    EAPSITES_output_table_name,
                                    EAPSITES_output_fc_path,
                                    address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc")

#### Reference Cells

In [ ]:
# Leads Processing for 4/17/2020
LEADS_input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\Phase1_ARC_Sample_File_4.xlsx"
LEADS_input_sheet_name = "Sheet1"
LEADS_address_field = "Address"
LEADS_city_field = "City"
LEADS_state_field = r"State/Province/Region"
LEADS_zipcode_field = r"Zipcode"
LEADS_output_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_Leads_200417_v2"
LEADS_output_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_Leads_200417_v2"

# Collection Site processing for 4/13/2020
BCS_input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Archived\Phase1_ARC_Sample_File_3.xlsx"
BCS_input_sheet_name = "CollectionSites"
BCS_address_field = "AddressLine1"
BCS_city_field = "City"
BCS_state_field = "State"
BCS_zipcode_field = "ZipCode"
BCS_output_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_CollectionSites_200413_v2"
BCS_output_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_CollectionSites_200413_v2"

# Leads to BCS Connections
workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
routing_nd = r"C:\ArcGIS\Business Analyst\US_2019\Data\Streets Data\NorthAmerica.gdb\Routing\Routing_ND"
CONN_output_connections_name = "Connections_Leads_to_BCS_200417_V2"

# Post-process Connections
connections_fc = "Connections_Leads_to_BCS_200417_V2"
leads_fc = "FC_Leads_200417_V2"

# Post-process BCS Summary Stats
connections_fc = "Connections_Leads_to_BCS_200417_V2"
facilities_fc = "FC_CollectionSites_200413_V2"
leads_fc = "FC_Leads_200417_V2"
output_summary_table = "in_memory\SummaryStats"
output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_facilities_fc_name = "FC_CollectionSites_200413_V2_Processed"

# Post-process Leads Summary Stats
connections_fc = "Connections_Leads_to_BCS_200417_V2"
leads_fc = "FC_Leads_200417_V2"
output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_leads_fc_name = "FC_Leads_200417_V2_Processed"

In [ ]:
# # Leads Processing for 4/17/2020

# input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\Phase1_ARC_Sample_File_4.xlsx"
# input_sheet_name = "Sheet1"
# address_field = "Address"
# city_field = "City"
# state_field = r"State/Province/Region"
# zipcode_field = r"Zipcode"
# output_facilities_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_Leads_200417_v2"
# output_facilities_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_Leads_200417_v2"

# incidents_fc = process_incidents(input_excel, 
#                                  input_sheet_name,
#                                  address_field,
#                                  city_field,
#                                  state_field,
#                                  zipcode_field,
#                                  output_facilities_table_path,
#                                  output_facilities_fc_path,
#                                  address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
#                                  incident_field=None,
#                                  symbology_layer=None)

In [ ]:
# # Leads to BCS Connections
# workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
# output_connections_name = "Connections_Leads_to_BCS_200417_V2"
# routing_nd = r"C:\ArcGIS\Business Analyst\US_2019\Data\Streets Data\NorthAmerica.gdb\Routing\Routing_ND"

# process_connections(facilities_fc, 
#                     incidents_fc,
#                     workspace_fgdb,
#                     output_connections_name,
#                     routing_nd)

In [ ]:
# # Post-process Connections
# connections_fc = "Connections_Leads_to_BCS_200417_V2"
# leads_fc = "FC_Leads_200417_V2"
# post_process_connections_with_lead_info(connections_fc, leads_fc)

In [ ]:
# # Post-process BCS Summary Stats

# connections_fc = "Connections_Leads_to_BCS_200417_V2"
# facilities_fc = "FC_CollectionSites_200413_V2"
# leads_fc = "FC_Leads_200417_V2"
# output_summary_table = "in_memory\SummaryStats"
# output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
# output_facilities_fc_name = "FC_CollectionSites_200413_V2_Processed"

# post_process_facilities_with_connection_info(connections_fc,
#                                              facilities_fc,
#                                              output_summary_table,
#                                              output_fgdb,
#                                              output_facilities_fc_name)

In [ ]:
# # Post-process Leads Summary Stats
# connections_fc = "Connections_Leads_to_BCS_200417_V2"
# leads_fc = "FC_Leads_200417_V2"
# output_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
# output_leads_fc_name = "FC_Leads_200417_V2_Processed"
# leads_fc = post_process_incidents_with_facilities(leads_fc, connections_fc, output_fgdb, output_leads_fc_name)

### Phase 2: EAPSites to EAPNeeds

In [ ]:
# EAPSite processing for 4/17/2020

input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\Phase2_ARC_Sample_File_1.xlsx"
input_sheet_name = "EAPSites"
address_field = "Address"
city_field = "City"
state_field = r"State"
zipcode_field = r"ZipCode"
output_facilities_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_EAPSites_200417"
output_facilities_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_EAPSites_200417"

facilities_fc = process_facilities(input_excel, 
                                   input_sheet_name,
                                   address_field,
                                   city_field,
                                   state_field,
                                   zipcode_field,
                                   output_facilities_table_path,
                                   output_facilities_fc_path,
                                   address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                                   facility_field=None,
                                   symbology_layer=None)

In [ ]:
# EAPNeed Processing for 4/17/2010

input_excel = r"C:\Users\albe9057\Documents\4_Sandbox_Projects\2004_ARC_COVID19_Plasma\Data\Operational\Phase2_ARC_Sample_File_1.xlsx"
input_sheet_name = "EAPNeed"
address_field = "Address"
city_field = "City"
state_field = r"State"
zipcode_field = r"ZipCode"
output_facilities_table_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\Table_EAPNeed_200417_v2"
output_facilities_fc_path = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb\FC_EAPNeed_200417_v2"

incidents_fc = process_incidents(input_excel, 
                                 input_sheet_name,
                                 address_field,
                                 city_field,
                                 state_field,
                                 zipcode_field,
                                 output_facilities_table_path,
                                 output_facilities_fc_path,
                                 address_locator=r"C:\ArcGIS\Business Analyst\US_2019\Data\Geocoding Data\USA_LocalComposite.loc",
                                 incident_field=None,
                                 symbology_layer=None)

In [ ]:
# EAP Connections
workspace_fgdb = r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\ARC_Convalescent_Plasma.gdb"
output_connections_name = "Connections_EAPNeeds_to_EAPSites_200417"
routing_nd = r"C:\ArcGIS\Business Analyst\US_2019\Data\Streets Data\NorthAmerica.gdb\Routing\Routing_ND"

process_connections(facilities_fc, 
                    incidents_fc,
                    workspace_fgdb,
                    output_connections_name,
                    routing_nd)

## Reference Code

In [ ]:
arcpy.management.ApplySymbologyFromLayer("CollectionSites_FC_200413", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\CollectionSiteslyrx.lyrx", None, "DEFAULT")

In [ ]:
arcpy.management.DeleteField("CollectionSites_FC_200413", "AddNum;AddNumFrom;AddNumTo;Addr_type;IN_Address;City;City;Country;Country;DisplayX;DisplayY;Distance;LangCode;Loc_name;Match_addr;Match_type;Postal;Rank;Region;RegionAbbr;Score;Side;StAddr;IN_State;Xmax;Xmin;Ymax;Ymin")

In [ ]:
arcpy.management.ApplySymbologyFromLayer("DonorLeads_FC_200413", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\DonorLeadslyrx.lyrx", "VALUE_FIELD USER_Likely USER_Likely", "MAINTAIN")

In [ ]:
arcpy.management.DeleteField("DonorLeads_FC_200413", "AddNum;AddNumFrom;AddNumTo;Addr_type;IN_Address;City;City;Country;DisplayX;DisplayY;Distance;LangCode;Loc_name;Match_addr;Match_type;USER_Postal___Zip_Code;Rank;Region;RegionAbbr;Score;Side;StAddr;USER_State___Province___Region;Status;StDir;StName;StPreDir;StPreType;StType;Subregion;Xmax;Xmin;Ymax;Ymin")

In [ ]:
arcpy.management.ApplySymbologyFromLayer("Routes_FC_AllDLtoBCS_200413", r"C:\Users\albe9057\Documents\ArcGIS\Projects\ARC_Convalescent_Plasma\Connections_DonorstoCollectionSites.lyrx", None, "MAINTAIN")

C1. Collection Centers to Hospitals

## D. Publish Outputs

## E. Clustering Analysis

## F. Threshold Analysis